In [1]:
%%capture
!pip install selenium
!pip install chromedriver-autoinstaller
!pip install jupyternotify

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from chromedriver_autoinstaller import install as chromedriver
from time import sleep
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [3]:
try:
    file = open('../eduro_userdata', 'r')
except FileNotFoundError:
    driver = webdriver.Chrome(executable_path=chromedriver())
    driver.get('https://hcs.eduro.go.kr/')
    while True:
        users = driver.execute_script('''return localStorage.getItem('users');''')
        password = driver.execute_script('''return document.querySelector('input[type="password"]')?.value;''')
        if users:
            file = open('../eduro_userdata', 'w')
            file.write('\n'.join([users, password]))
            file.close()
            break
        sleep(0.1)
    driver.close()
else:
    users, password = file.read().split('\n')
    file.close()

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_argument('disable-infobars')
options.add_argument('--disable-extensions')

driver = webdriver.Chrome(executable_path=chromedriver(), options=options)
driver.get('https://hcs.eduro.go.kr/')

wait = WebDriverWait(driver, 10)

In [5]:
driver.execute_script(f'''localStorage.setItem('users', '{users}');''')
driver.refresh()

In [6]:
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="password"]'))).send_keys(password)
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#btnConfirm'))).click()

wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.btn'))).click()

wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[id$="a1"]')))
answers = driver.find_elements(By.CSS_SELECTOR, 'input[id$="a1"]')
for answer in answers:
    answer.click()
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[type="submit"]'))).click()

%notify -m "자가진단을 완료했습니다."

<IPython.core.display.Javascript object>

In [7]:
driver.close()